In [ ]:
import h5py
import numpy as np
import torch
import scipy.signal as signal
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
from torch.utils.data import Dataset, DataLoader

# Load the RF singal IQ data


# Convert from 32bit float sample by interleaving real and imaginary parts into 64bit complex numbers
def hdf5_to_dataset(hdf5_file):
    """
    Load the IQ data from the HDF5 file
    """
    source_data = h5py.File(hdf5_file, "r")
    source_keys = list(source_data.keys())

    return source_data, source_keys

def interleaved_to_complex(source_data, source_key):
    """
    Convert interleaved IQ data to complex numbers
    """
    source_data = source_data.get(source_key)
    source_data = np.array(source_data) 
    source_data = source_data / 32768
    source_data = source_data.astype(np.float32).view(np.complex64)

    return source_data

def iq_to_psd(iq_data, num_fft, sample_rate):
    """
    Compute the PSD of the input IQ data
    """
    fft_vals = torch.fft.fft(iq_data, n=num_fft)
    psd = (fft_vals.abs() ** 2) / (sample_rate * num_fft)

    return psd

def render_iq_psd(source_data, source_keys):
    for key in source_keys:
        I = source_data[::10].real
        Q = source_data[::10].imag
        colors = plt.cm.viridis(np.linspace(0, 1, len(I)))
        plt.figure(figsize=(8, 6))
        plt.scatter(I, Q, c=colors, s=10)
        plt.xlabel('In-Phase (I)')
        plt.ylabel('Quadrature (Q)')
        plt.title('Constellation Diagram of IQ Data')
        plt.show()
        plt.psd(source_data, NFFT=2048, Fs=30000000, Fc=2437000000)
        plt.title(f'PSD for key: {key}')
        plt.show()
        input("Press Enter to continue...")

: 

In [ ]:
class RFSignalDataset(Dataset):
    def __init__(self, source_data, source_keys):
        """
        Initialize the dataset with the provided data.
        :param data: Numpy array containing the RF signal data
        """
        self.data = []
        for key in source_keys:
            data = interleaved_to_complex(source_data, key)
            print("loaded " + key)
            self.data.append(data)
        self.data = np.concatenate(self.data)

    def __len__(self):
        """
        Return the total number of samples in the dataset.
        """
        return len(self.data)

    def __getitem__(self, idx):
        """
        Fetch the data sample at the specified index.
        :param idx: Index of the data sample
        :return: The data sample as a torch tensor
        """
        sample = torch.tensor(self.data[idx], dtype=torch.complex64)
        return sample

# Create the dataset and DataLoader
source_data, source_keys = hdf5_to_dataset("2_4ghz_indoor.h5")
print(source_keys)
rf_dataset = RFSignalDataset(source_data=source_data, source_keys=source_keys)
data_loader = DataLoader(rf_dataset, batch_size=24, shuffle=True)

# Example: Accessing a batch from the data_loader
for batch in data_loader:
    print(batch.shape)  # Should be [batch_size]
    break